In [1]:
import numpy as np
import pandas as pd
X = pd.read_csv('C:\\Users\\usmanzafar\\Desktop\\refinedperm.csv')
# limit to categorical data using df.select_dtypes()
X = X.select_dtypes(include=[object])
X['pw_amount_9089'] = X['pw_amount_9089'].str.replace(',', '')
X=X.dropna(how='any') 
X.shape
X=X.head(10000)

C:\Users\usmanzafar\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [3]:
# TODO: create a LabelEncoder object and fit it to each feature in X
# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
X_2 = X.apply(le.fit_transform)
X_2.head()

,case_status,class_of_admission,decision_date,employer_address_1,employer_city,employer_name,employer_postal_code,employer_state,job_info_work_city,job_info_work_state,pw_amount_9089,pw_job_title_9089,pw_soc_code,pw_soc_title,pw_source_name_9089,pw_unit_of_pay_9089
98304,1,14,88,4762,166,3110,277,22,206,22,240,267,52,267,2,3
98305,1,14,97,4040,121,1130,815,9,160,9,1872,1,28,1,2,3
98306,1,21,102,4731,990,2798,2440,49,1233,49,2579,264,51,264,2,3
98307,1,14,94,886,528,4921,392,48,1175,45,2471,163,56,163,2,3
98308,1,14,82,131,925,793,1407,14,982,14,3727,267,52,267,2,3


In [4]:
X_2 = X_2.reset_index(drop=True)

In [5]:
X_2.head(10)

,case_status,class_of_admission,decision_date,employer_address_1,employer_city,employer_name,employer_postal_code,employer_state,job_info_work_city,job_info_work_state,pw_amount_9089,pw_job_title_9089,pw_soc_code,pw_soc_title,pw_source_name_9089,pw_unit_of_pay_9089
0,1,14,88,4762,166,3110,277,22,206,22,240,267,52,267,2,3
1,1,14,97,4040,121,1130,815,9,160,9,1872,1,28,1,2,3
2,1,21,102,4731,990,2798,2440,49,1233,49,2579,264,51,264,2,3
3,1,14,94,886,528,4921,392,48,1175,45,2471,163,56,163,2,3
4,1,14,82,131,925,793,1407,14,982,14,3727,267,52,267,2,3
5,1,14,89,159,572,4703,2017,31,696,31,3258,73,49,73,2,3
6,0,14,116,1789,1067,2181,2360,4,653,38,3153,187,86,187,2,3
7,1,14,77,183,176,2968,229,39,240,39,3536,264,51,264,2,3
8,1,14,84,211,288,4126,2353,4,350,4,2586,72,55,72,2,3
9,1,14,89,115,747,1507,1191,51,997,31,3316,264,51,264,2,3


In [6]:
df_glass=X_2

In [7]:
def get_train_test(df, y_col, ratio):
    mask = np.random.rand(len(df)) < ratio
    df_train = df[mask]
    df_test = df[~mask]
    
    Y_train = df_train[y_col].values
    Y_test = df_test[y_col].values
    del df_train[y_col]
    del df_test[y_col]
 
    X_train = df_train.values
    X_test = df_test.values
    return X_train, Y_train, X_test, Y_test
 
y_col = 'case_status'
train_test_ratio = 0.7
X_train, Y_train, X_test, Y_test = get_train_test(df_glass, y_col, train_test_ratio)

In [8]:
dict_classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Nearest Neighbors": KNeighborsClassifier(),
    "Linear SVM": SVC(),
    "Gradient Boosting Classifier": GradientBoostingClassifier(),
    "Decision Tree": tree.DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators = 18),
    "Neural Net": MLPClassifier(alpha = 1),
    "Naive Bayes": GaussianNB()
}

In [9]:
no_classifiers = len(dict_classifiers.keys())
 
def batch_classify(X_train, Y_train, X_test, Y_test, verbose = True):
    df_results = pd.DataFrame(data=np.zeros(shape=(no_classifiers,4)), columns = ['classifier', 'train_score', 'test_score', 'training_time'])
    count = 0
    for key, classifier in dict_classifiers.items():
        t_start = time.clock()
        classifier.fit(X_train, Y_train)
        t_end = time.clock()
        t_diff = t_end - t_start
        train_score = classifier.score(X_train, Y_train)
        test_score = classifier.score(X_test, Y_test)
        df_results.loc[count,'classifier'] = key
        df_results.loc[count,'train_score'] = train_score
        df_results.loc[count,'test_score'] = test_score
        df_results.loc[count,'training_time'] = t_diff
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=key, f=t_diff))
        count+=1
    return df_results

In [10]:
df_results = batch_classify(X_train, Y_train, X_test, Y_test)
print("Results of algorithm with only 30 and 70 percent test and training data split!")
print(df_results.sort_values(by='test_score', ascending=False))

trained Logistic Regression in 0.27 s
trained Nearest Neighbors in 0.04 s
trained Linear SVM in 5.35 s
trained Gradient Boosting Classifier in 0.29 s
trained Decision Tree in 0.01 s
trained Random Forest in 0.10 s
trained Neural Net in 0.63 s
trained Naive Bayes in 0.00 s
Results of algorithm with only 30 and 70 percent test and training data split!
                     classifier  train_score  test_score  training_time
3  Gradient Boosting Classifier     1.000000    1.000000       0.286672
4                 Decision Tree     1.000000    1.000000       0.006455
5                 Random Forest     1.000000    1.000000       0.097025
7                   Naive Bayes     0.983061    0.982202       0.003391
0           Logistic Regression     0.971863    0.971655       0.266176
6                    Neural Net     0.930807    0.911668       0.632784
2                    Linear SVM     0.999713    0.626236       5.354556
1             Nearest Neighbors     0.767298    0.620962       0.040155
